<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/RouteLLM_Summarization_RAG_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install llama-index==0.12.9 llama-index-llms-ollama==0.5.0 llama-index-embeddings-huggingface==0.4.0 llama-index-llms-groq==0.3.1

In [33]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

In [34]:
import nest_asyncio
nest_asyncio.apply()

In [35]:
from llama_index.core import SimpleDirectoryReader
documents= SimpleDirectoryReader(input_files=["/content/R22B.Tech.RevisedAcademicRegulations.pdf"]).load_data()

In [36]:
documents

[Document(id_='11aa754d-b107-4266-b627-0252f969684e', embedding=None, metadata={'page_label': '1', 'file_name': 'R22B.Tech.RevisedAcademicRegulations.pdf', 'file_path': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'file_type': 'application/pdf', 'file_size': 798830, 'creation_date': '2025-02-04', 'last_modified_date': '2025-02-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \n1  \n   JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY HYDERABAD (Established by State Act No. 30 of 2008) Kukatpally, Hyderabad, Telangana (India).  ACADEMIC REGULATIONS (R22) FOR B.TECH REGULAR STUDENTS  WITH EFFECT FROM THE ACADEMIC YEAR 2022-23   1.0 Und

In [37]:
len(documents)

25

In [38]:
documents[0]

Document(id_='11aa754d-b107-4266-b627-0252f969684e', embedding=None, metadata={'page_label': '1', 'file_name': 'R22B.Tech.RevisedAcademicRegulations.pdf', 'file_path': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'file_type': 'application/pdf', 'file_size': 798830, 'creation_date': '2025-02-04', 'last_modified_date': '2025-02-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' \n1  \n   JAWAHARLAL NEHRU TECHNOLOGICAL UNIVERSITY HYDERABAD (Established by State Act No. 30 of 2008) Kukatpally, Hyderabad, Telangana (India).  ACADEMIC REGULATIONS (R22) FOR B.TECH REGULAR STUDENTS  WITH EFFECT FROM THE ACADEMIC YEAR 2022-23   1.0 Unde

In [39]:
from llama_index.core.node_parser import SentenceSplitter

In [40]:
splitter=SentenceSplitter(chunk_size=1000,chunk_overlap=50)

In [41]:
nodes=splitter.get_nodes_from_documents(documents)

In [42]:
print(type(nodes))

<class 'list'>


In [43]:
nodes

[TextNode(id_='ca62cbd5-fc7b-4e05-b5e8-b07366d55cc3', embedding=None, metadata={'page_label': '1', 'file_name': 'R22B.Tech.RevisedAcademicRegulations.pdf', 'file_path': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'file_type': 'application/pdf', 'file_size': 798830, 'creation_date': '2025-02-04', 'last_modified_date': '2025-02-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='11aa754d-b107-4266-b627-0252f969684e', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'R22B.Tech.RevisedAcademicRegulations.pdf', 'file_path': '/content/R22B.Tech.RevisedAcademicRegulations.pdf', 'file_type': 'application/pdf', 'file_size': 798830, 'creation_date': '2025-02-04', 'last_modifi

In [44]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [59]:
Settings.llm=Groq(model="llama-3.3-70b-versatile")

In [60]:
Settings.embed_model=HuggingFaceEmbedding()

In [61]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index=SummaryIndex(nodes)
vector_index=VectorStoreIndex(nodes)

In [62]:
from urllib.parse import uses_params
summary_query_engine=summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine=vector_index.as_query_engine()

In [63]:
from llama_index.core.tools import QueryEngineTool

summary_tool=QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization tasks related to given context"),
)

vector_tool=QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieve specific context information related to given question"),
)

In [64]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine=RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool,vector_tool],
    verbose=True
)

In [65]:
response = query_engine.query("Summarize the given document")
print(response)

Selecting query engine 0: The question asks to summarize the given document, which aligns with option 1, useful for summarization tasks related to given context..
The document outlines the academic regulations for undergraduate students in a B.Tech program. It covers various aspects such as eligibility, program structure, course registration, attendance requirements, and academic requirements. The program is a 4-year undergraduate degree offered under the Choice Based Credit System, requiring students to complete 160 credits with a minimum CGPA of 5. The regulations specify the course registration process, attendance requirements, and academic requirements, including evaluation patterns and project work. Students must meet specific credit requirements to progress from one semester to the next and must score at least 35% in Continuous Internal Evaluation and 35% in Semester End Examinations to pass a subject. The document also outlines the grading system, passing standards, and conditio

In [68]:
response = query_engine.query("About which college is mentioned here?")
print(response)

Selecting query engine 1: The question 'About which college is mentioned here?' requires retrieving specific context information related to the given question, which aligns with choice 2..
Jawaharlal Nehru Technological University Hyderabad.
